# CIAN.ru

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

In [4]:
page = 1

### Для ЦАО находим главную ссылку и в ней находим **&p=1&**, это и есть переход по страницам

In [5]:
district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

In [6]:
district

'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

In [182]:
page_url =  district.format(page)

search_page = requests.get(page_url)
search_page = search_page.content
search_page = BeautifulSoup(search_page, 'lxml')


#число квартир в округе, просто ради интереса
#хотелось собрать все, но слишком долго

flat_num = str(search_page.find('div', attrs={'class':'serp-above__count'}).contents[1])
flat_num = int(re.findall('\d{1,}', flat_num)[0])
flat_num

24390

### Гуляем по страницам и качаем с них все ссылки на наши квартиры

In [184]:
links = []
#page_num = int(flat_num/31)
for page in range(1, 30):
    page_url =  district.format(page)

    search_page = requests.get(page_url)
    search_page = search_page.content
    search_page = BeautifulSoup(search_page, 'lxml')

    flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
    flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

    for link in flat_urls:
        if link.isdigit():
            links.append(link)

In [185]:
links

['149365299',
 '150122016',
 '149914273',
 '149031353',
 '149031259',
 '151004941',
 '150346368',
 '151031275',
 '151004981',
 '149031321',
 '148794719',
 '150640194',
 '149565205',
 '151031063',
 '149849414',
 '145609912',
 '149300855',
 '149788538',
 '148112883',
 '149727197',
 '151005044',
 '150463857',
 '151005116',
 '148800467',
 '151005216',
 '151005085',
 '149031349',
 '140980631',
 '148721148',
 '148142909',
 '144872188',
 '149031376',
 '150459322',
 '150678964',
 '150679023',
 '150977773',
 '150720249',
 '147655377',
 '149431030',
 '149213362',
 '150411534',
 '19220848',
 '149031338',
 '150684424',
 '150519279',
 '149213364',
 '149031293',
 '150645957',
 '150974485',
 '150545360',
 '149808503',
 '150118457',
 '148829580',
 '150833956',
 '149031370',
 '150627596',
 '144380578',
 '139573007',
 '150125217',
 '150977807',
 '149396229',
 '149311291',
 '149031368',
 '150684509',
 '150684571',
 '149390159',
 '149063476',
 '149031336',
 '149031388',
 '150945577',
 '149507258',
 '14950

In [186]:
len(links)

812

Итого вытащили 812 квартирок по ЦАО

### Попробуем перейти на первую

In [354]:
flat_url = 'http://www.cian.ru/sale/flat/' + str(links[1]) + '/'
#flat_url = 'http://www.cian.ru/sale/flat/150531912/' 
flat_page = requests.get(flat_url)
flat_page = flat_page.content
flat_page = BeautifulSoup(flat_page, 'lxml')

In [355]:
flatStats = {'District':0}

In [356]:
print(flat_url)

http://www.cian.ru/sale/flat/150122016/


# Цена

In [357]:
price = flat_page.find('div', attrs={'class':'object_descr_price'})
price = re.split('<div>|за м<sup>2</sup>|\n|руб|\W', str(price))

In [358]:
print(price)

['', 'div', 'class', '', 'object_descr_price', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '1', '200', '000', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'span', 'style', '', 'color', '', '', '808080', '', '', '', '', '76', '483', '680', '', '', '', 'span', '', '', '', 'div', '']


In [359]:
[i for i in price if i.isdigit()][-3:]

['76', '483', '680']

In [360]:
"".join([i for i in price if i.isdigit()][-3:])

'76483680'

In [361]:
price = "".join([i for i in price if i.isdigit()][-3:])

In [362]:
def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = "".join([i for i in price if i.isdigit()][-3:])
    return int(price)

In [363]:
getPrice(flat_page)

76483680

In [364]:
flatStats['Price'] = getPrice(flat_page)

# Координаты

In [365]:
flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]

<a href="http://map.cian.ru/?deal_type=2&amp;flats=yes&amp;center=55.721731%2C37.572899&amp;room0=1&amp;room1=1&amp;room2=1&amp;room3=1" rel="nofollow" target="_blank"><i></i>Посмотреть предложения рядом</a>

In [366]:
coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
coords = re.split('&amp|center=|%2C', str(coords))
coords_list = []
for item in coords:
    if item[0].isdigit():
        coords_list.append(item)
lat = float(coords_list[0])
lon = float(coords_list[1])

In [367]:
lat, lon

(55.721731, 37.572899)

In [412]:
def getCoords(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0])
    lon = float(coords_list[1])
    return lon, lat

In [413]:
getCoords(flat_page)

(37.572899, 55.721731)

In [370]:
flatStats['lat'], flatStats['lon'] = getCoords(flat_page)

# Число комнат

In [371]:
rooms = flat_page.find('div', attrs={'class':'object_descr_title'})

In [372]:
rooms

<div class="object_descr_title" itemprop="name">
                
    
        3-комн. кв. 
    

                
                    
                        в <a href="/zhiloy-kompleks-clerkenwell-house-moskva-5341/" target="_blank">ЖК «Clerkenwell House»</a>
</div>

In [373]:
rooms = html_stripper(rooms)

In [374]:
rooms

'\n                \n    \n        3-комн. кв.\xa0\n    \n\n                \n                    \n                        в\xa0ЖК «Clerkenwell House»\n'

In [375]:
room_number = ''
for i in re.split('-|\n', rooms):
    if 'комн' in i:
        break
    else:
        room_number += i
room_number = "".join(room_number.split())

In [376]:
room_number

'3'

In [377]:
def getRoom(flat_page):
    rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
    rooms = html_stripper(rooms)
    room_number = ''
    for i in re.split('-|\n', rooms):
        if 'комн' in i:
            break
        else:
            room_number += i
    room_number = "".join(room_number.split())
    return room_number

In [378]:
getRoom(flat_page)

'3'

In [379]:
flatStats['rooms'] = getRoom(flat_page)

# Таблица

In [380]:
table = flat_page.find('table', attrs = {'class':'object_descr_props'})

In [381]:
table

<table class="object_descr_props flat sale" style="float:left">
<tr>
<th style="padding-top:5px;"> # Общая информация:</th>
</tr>
<tr>
<th>Статус помещения:</th>
<td>апартаменты</td>
</tr>
<tr>
<th>Этаж:</th>
<td>
                3 / 5
                
            </td>
</tr>
<tr>
<th>Тип дома:</th>
<td>
            вторичка
            
        </td>
</tr>
<tr>
<th>Тип продажи:</th>
<td>свободная</td>
</tr>
<tr>
<th>Общая площадь:</th>
<td>
<i class="object_descr_details_color"></i>95 м<sup>2</sup>
</td>
</tr>
<tr>
<th>Площадь комнат:</th>
<td>
<i class="object_descr_details_color"></i><!--
-->0 м<sup>2</sup>
</td>
</tr>
<tr>
<th>Жилая площадь:</th>
<td>
<i class="object_descr_details_color" style="background-color:#90C090"></i>65 м<sup>2</sup>
</td>
</tr>
<tr>
<th>Площадь кухни:</th>
<td>
<i class="object_descr_details_color" style="background-color:#C09090"></i>–
        </td>
</tr>
<tr>
<th>Совмещенных санузлов:</th>
<td><i class="object_descr_details_color" style="background-color

In [382]:
print(html_stripper(table))



 # Общая информация:


Статус помещения:
апартаменты


Этаж:

                3 / 5
                
            


Тип дома:

            вторичка
            
        


Тип продажи:
свободная


Общая площадь:

95 м2



Площадь комнат:

0 м2



Жилая площадь:

65 м2



Площадь кухни:

–
        


Совмещенных санузлов:
2
            


Балкон:
–
        


Лифт:
1 пасс.


Парковка:
подземная


Телефон:
да


Вид из окна:
улица


Ремонт:
дизайнерский




In [383]:
table = html_stripper(table)

In [384]:
print(re.split('Этаж|/', table)[1])

:

                3 


In [385]:
flat_url

'http://www.cian.ru/sale/flat/150122016/'

In [386]:
flatStats

{'District': 0,
 'Price': 76483680,
 'lat': 55.721731,
 'lon': 37.572899,
 'rooms': '3'}

# Этаж
Общая идея заключается в том, чтобы с помощью регулярных выражений выделить из таблицы численное значение, заключенное между текстом.

In [387]:
floor = re.split('Этаж|/', table)[1]
floor = re.findall('\d{1,}', floor)[0]
floor = int(floor)
print(floor)

3


In [388]:
def getFloor(table):
    floor = re.split('Этаж|/', table)[1]
    floor = re.findall('\d{1,}', floor)[0]
    return int(floor)

print(getFloor(table))

3


In [389]:
table

'\n\n # Общая информация:\n\n\nСтатус помещения:\nапартаменты\n\n\nЭтаж:\n\n                3\xa0/\xa05\n                \n            \n\n\nТип дома:\n\n            вторичка\n            \n        \n\n\nТип продажи:\nсвободная\n\n\nОбщая площадь:\n\n95\xa0м2\n\n\n\nПлощадь комнат:\n\n0\xa0м2\n\n\n\nЖилая площадь:\n\n65\xa0м2\n\n\n\nПлощадь кухни:\n\n–\n        \n\n\nСовмещенных санузлов:\n2\n            \n\n\nБалкон:\n–\n        \n\n\nЛифт:\n1 пасс.\n\n\nПарковка:\nподземная\n\n\nТелефон:\nда\n\n\nВид из окна:\nулица\n\n\nРемонт:\nдизайнерский\n\n'

# Общая площадь

In [390]:
sq = re.split('Общая площадь|м2', table)[1]
sq = re.findall('\d{1,}', sq)[0]
sq = re.sub(',','.', sq)
sq = float(sq)
print(sq)

95.0


In [391]:
def getTotSp(table):
    try:
        sq = re.split('Общая площадь|Площадь комнат', table)[1]
        sq = re.findall('\d{1,}', sq)[0]
        sq = re.sub(',','.', sq)
    except:
        sq = 0
    return float(sq)

print(getTotSp(table))

95.0


# Жилая площадь

In [392]:
def getLivSp(table):
    try:
        liv = re.split('Жилая площадь|Площадь кухни', table)[1]
        liv = re.findall('\d{1,}', liv)[0]
        liv = re.sub(',','.', liv)
    except:
        liv = 0
    return float(liv)

print(getLivSp(table))

65.0


# Площадь кухни

In [446]:
def getKitSp(table):
    try:
        kit = re.split('Площадь кухни|Раздельных санузлов', table)[1]
        kit = re.findall('\d{1,}', kit)[0]
        kit = re.sub(',','.', kit)
    except:
        kit = 0
    return float(kit)

print(getKitSp(table))

1.0


# Расстояние до метро

Функция возвращает расстояние до метро в минутах и 1, если пешком, и 0, если на транспорте.

In [447]:
metrdist = str(flat_page.find('div', attrs={'class':'object_descr_metro'}))
metrdist = re.split('<span class="object_item_metro_comment">|</span>', metrdist)[2]
dist = int(re.findall('\d{1,}', metrdist)[0])
if re.findall('пешком', metrdist)[0] != None:  
    walk = 1
else:
    walk = 0
print(dist, walk)


6 1


In [448]:
def MetroDist(flat_page):
    try:
        metrdist = str(flat_page.find('div', attrs={'class':'object_descr_metro'}))
        metrdist = re.split('<span class="object_item_metro_comment">|</span>', metrdist)[2]
        dist = int(re.findall('\d{1,}', metrdist)[0])
        if re.findall('пешком', metrdist)[0] != None:  
            walk = 1
        else:
            walk = 0
    except:
        dist = 0
        walk = 0
    return int(dist), int(walk)

print(MetroDist(flat_page))
    

(6, 1)


# Телефон

In [449]:
try:
        telef = re.split('Телефон|Вид из окна', table)[1]
        if re.findall('да', telef)[0] != None:  
            tel = 1
except:
        tel = 0


In [450]:
def GetPhone(table):
    try:
        telef = re.split('Телефон|Вид из окна', table)[1]
        if re.findall('да', telef)[0] != None:  
            tel = 1
    except:
        tel = 0
    return int(tel)

print(GetPhone(table))
    

1


# Балкон

In [451]:
def GetBalcon(table):
    try:
        balc = re.split('Балкон|Лифт', table)[1]
        if re.findall('-|нет', balc)[0] != None:  
            bal = 1
    except:
        bal = 0
    return int(bal)

print(GetBalcon(table))

1


# Всего этажей

In [452]:
def getNFloor(table):
    Nfloor = re.split('/|Тип дома', table)[1]
    Nfloor = re.findall('\d{1,}', Nfloor)[0]
    return int(Nfloor)

print(getNFloor(table))


22


# Первичка или вторичка

In [533]:
#print(table)

In [461]:
def GetNew(table):
    try:
        new = re.split('Тип дома|Тип продажи', table)[1]
        if re.findall('[Вв]торичка', new)[0] != None:  
            new = 0
        else:
            new = 1
    except:
        new = 0
    return int(new)

print(GetNew(table))


0


# Расстояние от центра

Определим функцию, которая вычисляет [ортодромию](https://en.wikipedia.org/wiki/Great-circle_distance) - расстояние между двумя точками на поверхности Земли.


In [463]:
from math import radians, cos, sin, asin, sqrt


def haversine(lon, lat):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    lon_cent = 37.61981338262558
    lat_cent = 55.75370903771494 
  
    # convert decimal degrees to radians 
    lon_cent, lat_cent, lon, lat = map(radians, [lon_cent, lat_cent, lon, lat])

    # haversine formula 
    dlon = lon - lon_cent 
    dlat = lat - lat_cent 
    a = sin(dlat/2)**2 + cos(lat_cent) * cos(lat) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

lon, lat = getCoords(flat_page)
print(haversine(lon, lat))

4.079333572472215


### Создадим таблицу

* Rooms – число комнат
* Price – цена
* Total Space – общая площадь квартиры, кв. м.
* Living Space – жилая площадь квартиры, кв. м.
* Kitchen Space – площадь кухни, кв. м.
* City center distance – расстояние от центра в км. (можно по Пифагору)
* Metro distance – расстояние до метро в минутах
* Walk – 1 – если пешком от метро, 0 – если на транспорте
* Brick – 1 – кирпичный/монолит/жб, 0 – другой
* Telephone – 1 – если есть, 0 – нет
* Balcon – 1 – есть балкон/лоджия, 0 – нет
* Floor – номер этажа, на котором расположена квартира
* Floor Number – всего этажей в доме
* Market Type – 1 – первичный рынок, 0 - вторичка

In [464]:
import pandas as pd
CianFlats = pd.DataFrame(columns=['Rooms', 'Price', 'Total Space', 'Living Space', 'Kitchen Space', 'City center distance', 'Metro distance', 'Walk', 'Telephone', 'Balcon', 'Floor', 'Floor Number', 'Market Type'])
print(df.columns)


#заполним первую строку    
flat_url = 'http://www.cian.ru/sale/flat/' + str(links[3]) + '/'
flat_page = requests.get(flat_url)
flat_page = flat_page.content
flat_page = BeautifulSoup(flat_page, 'lxml')

table = flat_page.find('table', attrs = {'class':'object_descr_props'})
table = html_stripper(table)


lon, lat = getCoords(flat_page)
dist, walk = MetroDist(flat_page)
flatStats = pd.Series()
flatStats['Rooms'] = getRoom(flat_page)
flatStats['Price'] = getPrice(flat_page)
flatStats['Total Space'] = getTotSp(table)
flatStats['Living Space'] = getLivSp(table)
flatStats['Kitchen Space'] = getKitSp(table)
flatStats['City center distance'] = haversine(lon, lat)
flatStats['Metro distance'] = dist
flatStats['Walk'] = walk
flatStats['Telephone'] = GetPhone(table)
flatStats['Balcon'] = GetBalcon(table)
flatStats['Floor'] = getFloor(table)
flatStats['Floor Number'] = getNFloor(table)
flatStats['Market Type'] = GetNew(table)

flatStats


Index(['Rooms', 'Price', 'Total Space', 'Living Space', 'Kitchen Space',
       'City center distance', 'Metro distance', 'Walk', 'Telephone', 'Balcon',
       'Floor', 'Floor Number', 'Market Type'],
      dtype='object')


Rooms                         1
Price                   4403057
Total Space                  22
Living Space                 22
Kitchen Space                 1
City center distance    4.07933
Metro distance                6
Walk                          1
Telephone                     1
Balcon                        1
Floor                         3
Floor Number                 22
Market Type                   0
dtype: object

### Определим функцию, которая принимает ссылку на квартиру и возвращает строку таблицы

In [484]:
def getRow(link):
    flat_url = 'http://www.cian.ru/sale/flat/' + str(link) + '/'
    flat_page = requests.get(flat_url)
    flat_page = flat_page.content
    flat_page = BeautifulSoup(flat_page, 'lxml')

    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)


    lon, lat = getCoords(flat_page)
    dist, walk = MetroDist(flat_page)
    flatStats = pd.Series()
    flatStats['Rooms'] = getRoom(flat_page)
    flatStats['Price'] = getPrice(flat_page)
    flatStats['Total Space'] = getTotSp(table)
    flatStats['Living Space'] = getLivSp(table)
    flatStats['Kitchen Space'] = getKitSp(table)
    flatStats['City center distance'] = haversine(lon, lat)
    flatStats['Metro distance'] = dist
    flatStats['Walk'] = walk
    flatStats['Telephone'] = GetPhone(table)
    flatStats['Balcon'] = GetBalcon(table)
    flatStats['Floor'] = getFloor(table)
    flatStats['Floor Number'] = getNFloor(table)
    flatStats['Market Type'] = GetNew(table)
    
    return flatStats

print(getRow(links[0]))

CianFlats.append(getRow(links[0]), ignore_index=True)
#CianFlats.append(getRow(links[1]), ignore_index=True)
CianFlats

Rooms                          2
Price                   37500000
Total Space                   64
Living Space                  44
Kitchen Space                  8
City center distance      3.2702
Metro distance                 2
Walk                           1
Telephone                      0
Balcon                         0
Floor                          3
Floor Number                  11
Market Type                    0
dtype: object


,Rooms,Price,Total Space,Living Space,Kitchen Space,City center distance,Metro distance,Walk,Telephone,Balcon,Floor,Floor Number,Market Type


In [493]:
links
data = []
for i, link in enumerate(links):
    row = getRow(link)
    data.append(row)
print(data)

[Rooms                          2
Price                   37500000
Total Space                   64
Living Space                  44
Kitchen Space                  8
City center distance      3.2702
Metro distance                 2
Walk                           1
Telephone                      0
Balcon                         0
Floor                          3
Floor Number                  11
Market Type                    0
dtype: object, Rooms                          3
Price                   76483680
Total Space                   95
Living Space                  65
Kitchen Space                  2
City center distance     4.61182
Metro distance                10
Walk                           1
Telephone                      1
Balcon                         0
Floor                          3
Floor Number                   5
Market Type                    0
dtype: object, Rooms                          3
Price                   23000000
Total Space                   62
Living Space

In [499]:
CianFlats = CianFlats.append(data, ignore_index=True)
CianFlats.head()

,Rooms,Price,Total Space,Living Space,Kitchen Space,City center distance,Metro distance,Walk,Telephone,Balcon,Floor,Floor Number,Market Type
0,2,37500000.0,64.0,44.0,8.0,3.270199,2.0,1.0,0.0,0.0,3.0,11.0,0.0
1,3,76483680.0,95.0,65.0,2.0,4.611824,10.0,1.0,1.0,0.0,3.0,5.0,0.0
2,3,23000000.0,62.0,41.0,7.0,1.349326,6.0,1.0,1.0,0.0,2.0,3.0,0.0
3,1,4403057.0,22.0,22.0,1.0,4.079334,6.0,1.0,1.0,1.0,3.0,22.0,0.0
4,1,4585315.0,17.0,17.0,1.0,4.079334,6.0,1.0,1.0,1.0,2.0,17.0,0.0


### Сделаем таблицы для всех округов и соединим в одну

In [511]:
#Ссылки по округам

SAO = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=23&district%5B10%5D=33&district%5B11%5D=34&district%5B12%5D=35&district%5B13%5D=36&district%5B14%5D=37&district%5B15%5D=38&district%5B1%5D=24&district%5B2%5D=25&district%5B3%5D=26&district%5B4%5D=27&district%5B5%5D=28&district%5B6%5D=29&district%5B7%5D=30&district%5B8%5D=31&district%5B9%5D=32&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
SVAO = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=39&district%5B10%5D=49&district%5B11%5D=50&district%5B12%5D=51&district%5B13%5D=52&district%5B14%5D=53&district%5B15%5D=54&district%5B16%5D=55&district%5B1%5D=40&district%5B2%5D=41&district%5B3%5D=42&district%5B4%5D=43&district%5B5%5D=44&district%5B6%5D=45&district%5B7%5D=46&district%5B8%5D=47&district%5B9%5D=48&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
VAO = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=56&district%5B10%5D=66&district%5B11%5D=67&district%5B12%5D=68&district%5B13%5D=69&district%5B14%5D=70&district%5B15%5D=71&district%5B1%5D=57&district%5B2%5D=58&district%5B3%5D=59&district%5B4%5D=60&district%5B5%5D=61&district%5B6%5D=62&district%5B7%5D=63&district%5B8%5D=64&district%5B9%5D=65&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
UVAO = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=72&district%5B10%5D=82&district%5B11%5D=83&district%5B1%5D=73&district%5B2%5D=74&district%5B3%5D=75&district%5B4%5D=76&district%5B5%5D=77&district%5B6%5D=78&district%5B7%5D=79&district%5B8%5D=80&district%5B9%5D=81&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
UAO = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=84&district%5B10%5D=94&district%5B11%5D=95&district%5B12%5D=96&district%5B13%5D=97&district%5B14%5D=98&district%5B15%5D=99&district%5B1%5D=85&district%5B2%5D=86&district%5B3%5D=87&district%5B4%5D=88&district%5B5%5D=89&district%5B6%5D=90&district%5B7%5D=91&district%5B8%5D=92&district%5B9%5D=93&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
UZAO = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=100&district%5B10%5D=110&district%5B11%5D=111&district%5B1%5D=101&district%5B2%5D=102&district%5B3%5D=103&district%5B4%5D=104&district%5B5%5D=105&district%5B6%5D=106&district%5B7%5D=107&district%5B8%5D=108&district%5B9%5D=109&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
ZAO = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=112&district%5B10%5D=122&district%5B11%5D=123&district%5B12%5D=124&district%5B13%5D=348&district%5B14%5D=349&district%5B15%5D=350&district%5B1%5D=113&district%5B2%5D=114&district%5B3%5D=115&district%5B4%5D=116&district%5B5%5D=117&district%5B6%5D=118&district%5B7%5D=119&district%5B8%5D=120&district%5B9%5D=121&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
SZAO = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=125&district%5B1%5D=126&district%5B2%5D=127&district%5B3%5D=128&district%5B4%5D=129&district%5B5%5D=130&district%5B6%5D=131&district%5B7%5D=132&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
Zelik = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=152&district%5B1%5D=153&district%5B2%5D=154&district%5B3%5D=355&district%5B4%5D=356&district%5B5%5D=357&district%5B6%5D=358&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
NewMos = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=327&district%5B10%5D=337&district%5B1%5D=328&district%5B2%5D=329&district%5B3%5D=330&district%5B4%5D=331&district%5B5%5D=332&district%5B6%5D=333&district%5B7%5D=334&district%5B8%5D=335&district%5B9%5D=336&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
TAO = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=338&district%5B1%5D=339&district%5B2%5D=340&district%5B3%5D=341&district%5B4%5D=342&district%5B5%5D=343&district%5B6%5D=344&district%5B7%5D=345&district%5B8%5D=346&district%5B9%5D=347&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
CAO = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

### Объявим функцию, которая возвращает массив ссылок на квартиры для каждого округа

In [512]:
def getLinks(district):
    page = 1
    page_url =  district.format(page)

    search_page = requests.get(page_url)
    search_page = search_page.content
    search_page = BeautifulSoup(search_page, 'lxml')


    #число квартир в округе

    flat_num = str(search_page.find('div', attrs={'class':'serp-above__count'}).contents[1])
    flat_num = int(re.findall('\d{1,}', flat_num)[0])
    flat_num
    links = []
    #page_num = int(flat_num/31)
    for page in range(1, 30):
        page_url =  district.format(page)

        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')

        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

        for link in flat_urls:
            if link.isdigit():
                links.append(link)
    return links
links_SAO = getLinks(SAO)
links_SVAO = getLinks(SVAO)
links_VAO = getLinks(VAO)
links_UVAO = getLinks(UVAO)
links_UAO = getLinks(UAO)
links_UZAO = getLinks(UZAO)
links_ZAO = getLinks(ZAO)
links_SZAO = getLinks(SZAO)
links_Zelik = getLinks(Zelik)
links_NewMos = getLinks(NewMos)
links_TAO = getLinks(TAO)
links_CAO = getLinks(CAO)




In [514]:
len(links_TAO)

812

По 812 квартир в каждом округе

### Определим функцию, принимающую массив ссылок на квартиры определенного округа и возвращающую матрицу с данными

In [515]:
links_all = [links_CAO,links_SAO,links_SVAO,links_VAO,links_UVAO,links_UAO,links_UZAO,links_ZAO,links_SZAO,links_Zelik,links_NewMos,links_TAO]


#Определим функцию, принимающую массив ссылок на квартиры определенного округа и возвращающую матрицу с данными

def GetFinalTable(link):
    Flats = pd.DataFrame(columns=['Rooms', 'Price', 'Total Space', 'Living Space', 'Kitchen Space', 'City center distance', 'Metro distance', 'Walk', 'Telephone', 'Balcon', 'Floor', 'Floor Number', 'Market Type'])
    data = []
    for link in links:
        row = getRow(link)
        data.append(row)
    Flats = Flats.append(data, ignore_index=True)
    return Flats

#print(GetFinalTable(links_all[0]))
df = GetFinalTable(links_all[0])
CianFlats = CianFlats.append(df, ignore_index=True)
len(CianFlats)




5684

### Объявим датафрейм, в который соберем данные о квартирах со всех округов

In [517]:
#СОбъявим датафрейм, в который соберем данные о квартирах со всех округов

CianFlats_All = pd.DataFrame(columns=['Rooms', 'Price', 'Total Space', 'Living Space', 'Kitchen Space', 'City center distance', 'Metro distance', 'Walk', 'Telephone', 'Balcon', 'Floor', 'Floor Number', 'Market Type'])

for link in links_all:
    df = GetFinalTable(link)
    CianFlats_All = CianFlats_All.append(df, ignore_index=True)
    
len(CianFlats_All)

9744

In [530]:
#CianFlats_All.to_csv('CianFlats_All.csv')

In [532]:
CianFlats_All.head(10)

,Rooms,Price,Total Space,Living Space,Kitchen Space,City center distance,Metro distance,Walk,Telephone,Balcon,Floor,Floor Number,Market Type
0,2,37500000.0,64.0,44.0,8.0,3.270199,2.0,1.0,0.0,0.0,3.0,11.0,0.0
1,3,76483680.0,95.0,65.0,2.0,4.611824,10.0,1.0,1.0,0.0,3.0,5.0,0.0
2,3,23000000.0,62.0,41.0,7.0,1.349326,6.0,1.0,1.0,0.0,2.0,3.0,0.0
3,1,4403057.0,22.0,22.0,1.0,4.079334,6.0,1.0,1.0,1.0,3.0,22.0,0.0
4,1,4585315.0,17.0,17.0,1.0,4.079334,6.0,1.0,1.0,1.0,2.0,17.0,0.0
5,1,4814581.0,17.0,10.0,1.0,4.079334,6.0,1.0,1.0,0.0,2.0,5.0,0.0
6,1,4814581.0,17.0,17.0,1.0,4.126903,5.0,1.0,0.0,1.0,2.0,6.0,0.0
7,5,332704008.0,261.0,160.0,3.0,1.288559,5.0,1.0,0.0,0.0,3.0,7.0,0.0
8,1,5377566.0,18.0,10.0,1.0,4.079334,6.0,1.0,1.0,0.0,5.0,5.0,0.0
9,1,5377566.0,18.0,18.0,1.0,4.079334,6.0,1.0,1.0,1.0,5.0,18.0,0.0


Итого получилось 9744 квартиры